In [19]:
import numpy as np
import pandas as pd
import re
import string
import pickle

In [6]:
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

In [7]:
with open('../static/model/corpora/stopwords/english','r') as file:
    sw=file.read().splitlines()

In [8]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [10]:
def preprocessing(sentence):
    data = pd.DataFrame([sentence], columns=['Text'])
    data.loc[ :,"Text"] = data["Text"].apply(lambda x: " ".join(x.lower() for x in x.split()))
    data["Text"]=data["Text"].apply(lambda x: " ".join(re.sub(r'^https?:\/\/.*[\r\n]*', '', x,flags=re.MULTILINE) for x in x.split()))
    data["Text"]=data["Text"].apply(remove_punctuations)
    data["Text"] = data['Text'].str.replace(r'\d+', '', regex=True)
    data["Text"]=data["Text"].apply(lambda x: " ".join(x for x in x.split() if x not in sw))
    return data["Text"]

In [11]:
sentence = 'great product. i like it'

In [12]:
preprocessed_sentence = preprocessing(sentence)

In [13]:
preprocessed_sentence

0    great product like
Name: Text, dtype: object

In [14]:
vocab = pd.read_csv('../static/model/vocabulary.txt', header=None)
vocab_tokens = vocab[0].tolist()

In [15]:
vocab_tokens

['cant',
 'believ',
 'first',
 'review',
 'great',
 'productbr',
 'br',
 'use',
 'best',
 'organ',
 'infant',
 'formula',
 'iron',
 'ounc',
 'also',
 'product',
 'littl',
 'start',
 'show',
 'sign',
 'lactos',
 'intoler',
 'pediatrician',
 'search',
 'soy',
 'offer',
 'free',
 'altern',
 'mani',
 'prefer',
 'want',
 'natur',
 'nasti',
 'chemic',
 'ingredi',
 'consider',
 'amount',
 'research',
 'shop',
 'around',
 'found',
 'choic',
 'market',
 'one',
 'took',
 'well',
 'lot',
 'better',
 'find',
 'pack',
 'dollar',
 'less',
 'reason',
 'charg',
 'much',
 'month',
 'old',
 'origin',
 'spit',
 'st',
 'similac',
 'regular',
 'went',
 'sensit',
 'final',
 'trick',
 'past',
 'recal',
 'wife',
 'becam',
 'disgust',
 'suggest',
 'tri',
 'new',
 'brand',
 'anyway',
 'remind',
 'babi',
 'get',
 'older',
 'becom',
 'fussi',
 'mayb',
 'dont',
 'need',
 'anymor',
 'that',
 'local',
 'whole',
 'food',
 'store',
 'bought',
 'instant',
 'hit',
 'son',
 'love',
 'serious',
 'gobbl',
 'oz',
 'shot',
 

In [16]:
def vectorizer(ds, vocabulary):
    vectorized_list = []
    for sentence in ds:
        sentence_list = np.zeros(len(vocabulary))
        for i in range(len(vocabulary)):
            if vocabulary[i] in sentence.split():
                sentence_list[i] = 1
        vectorized_list.append(sentence_list)
    vectorized_list_new = np.asarray(vectorized_list, dtype=np.float32)
    return vectorized_list_new
        

In [17]:
vectorized_sentence = vectorizer(preprocessed_sentence, vocab_tokens)

In [18]:
vectorized_sentence

array([[0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [20]:
with open('../static/model/model.pickle', 'rb') as f:
    model = pickle.load(f)

In [22]:
model.predict(vectorized_sentence)

array([1], dtype=int64)

In [35]:
def get_prediction(vectorized_sentence):
    prediction = model.predict(vectorized_sentence)
    if prediction == 1:
        return 'good'
    else:
        return 'bad'

In [36]:
prediction_lable = get_prediction(vectorized_sentence)
prediction_lable

'good'

### Final code set needed

In [23]:
import numpy as np
import pandas as pd
import re
import string
import pickle

In [24]:
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

In [25]:
with open('../static/model/model.pickle', 'rb') as f:
    model = pickle.load(f)

In [26]:
with open('../static/model/corpora/stopwords/english','r') as file:
    sw=file.read().splitlines()

In [27]:
vocab = pd.read_csv('../static/model/vocabulary.txt', header=None)
vocab_tokens = vocab[0].tolist()

In [28]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [29]:
def preprocessing(sentence):
    data = pd.DataFrame([sentence], columns=['Text'])
    data.loc[ :,"Text"] = data["Text"].apply(lambda x: " ".join(x.lower() for x in x.split()))
    data["Text"]=data["Text"].apply(lambda x: " ".join(re.sub(r'^https?:\/\/.*[\r\n]*', '', x,flags=re.MULTILINE) for x in x.split()))
    data["Text"]=data["Text"].apply(remove_punctuations)
    data["Text"] = data['Text'].str.replace(r'\d+', '', regex=True)
    data["Text"]=data["Text"].apply(lambda x: " ".join(x for x in x.split() if x not in sw))
    return data["Text"]

In [30]:
def vectorizer(ds, vocabulary):
    vectorized_list = []
    for sentence in ds:
        sentence_list = np.zeros(len(vocabulary))
        for i in range(len(vocabulary)):
            if vocabulary[i] in sentence.split():
                sentence_list[i] = 1
        vectorized_list.append(sentence_list)
    vectorized_list_new = np.asarray(vectorized_list, dtype=np.float32)
    return vectorized_list_new

In [38]:
def get_prediction(vectorized_sentence):
    prediction = model.predict(vectorized_sentence)
    if prediction == 1:
        return 'good'
    else:
        return 'bad'

In [39]:
sentence = 'great product. i like it'
preprocessed_sentence = preprocessing(sentence)
vectorized_sentence = vectorizer(preprocessed_sentence, vocab_tokens)
prediction = model.predict(vectorized_sentence)
print(prediction)
prediction_lable = get_prediction(vectorized_sentence)
prediction_lable

[1]


'good'

In [40]:
sentence = 'bad product, i absolutely hate it.'
preprocessed_sentence = preprocessing(sentence)
vectorized_sentence = vectorizer(preprocessed_sentence, vocab_tokens)
prediction = model.predict(vectorized_sentence)
print(prediction)
prediction_lable = get_prediction(vectorized_sentence)
prediction_lable

[0]


'bad'

In [41]:
sentence = 'not interest, but i like it.'
preprocessed_sentence = preprocessing(sentence)
vectorized_sentence = vectorizer(preprocessed_sentence, vocab_tokens)
prediction = model.predict(vectorized_sentence)
print(prediction)
prediction_lable = get_prediction(vectorized_sentence)
prediction_lable

[0]


'bad'

In [42]:
sentence = 'i like to use it and it makes me lot of happiness.'
preprocessed_sentence = preprocessing(sentence)
vectorized_sentence = vectorizer(preprocessed_sentence, vocab_tokens)
prediction = model.predict(vectorized_sentence)
print(prediction)
prediction_lable = get_prediction(vectorized_sentence)
prediction_lable

[1]


'good'